# **Lab 2: Iterative methods**
**Jesper Lidaum**

# **Abstract**

In this report we implement 3 methods for solving equations and systems of equations. The first two are for solving systems of linear equations represented on matrix vector form. The last is for solving a real scalar equation. We implement Jordan itteration, Gauss-Seidel itteration and Newton's method.  

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2023 Jesper Lidbaum (jlidbaum@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [ ]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

This report implements 2 itterations described with mathematical formulas in the [course book](https://my.siam.org/Store/Product/viewproduct/?ProductId=39300058). And also the well known Newton's method which is described with psuedo code in the same book. 

## Jacobi iteration
The Jacobi iteration implemented in this report is based on chapter 7.7 in the course book. Jacobi iteration is related to Richardson iteration where we create the iterations matrix from two matricies $A=A_1+A_2$ where $A_1$ are the diagonal elements in $A$ and $A_2$ are the rest. From the course book we get the converge criterion $‖I−A_1^{-1}A‖<1$ for the iteration. And the iteration implemented in this lab can be found in example 7.8.

## Gauss-Seidel iteration
The Gauss-Seidel iteration is also related to the Richardson iteration. The iteration matrix is constructed by the upper triangular elements of $A$ and the rest. From example 7.9 in the course book we get the converge criterion $‖I−L^{−1}A‖<1$ where $L$ is the lower triangular elements of $A$. We also get the iteration that is implemented in this report from the same example.

## Newton's method
Newton's method is a method for solving nonlinear equations $f(x) = 0$. It takes the form $x^{(k+1)} = x^{(k)} - f(x^{(k)}) / f'(x^{(k)})$. The code in this report is based on algorithm 8.2 from the course book. 

# **Method**

In this part code for each of the algortihms are presented. The code is based on formulas and psuedo code from the course book.

## Jacobi iteration
The code is based on the iteration from example 7.8 in the course book. The stopping criterion is from chapter 7, page 145 where it is recomended to use that stopping criterion if we initialize with a zero vector. 

In [ ]:
def jacobi_iteration(A, b):

    TOL = 1e-6
    n = np.shape(b)[0]
    x = np.zeros(n)
    x_new = np.zeros(n)

    r = b - np.dot(A, x)

    while np.linalg.norm(r)/ np.linalg.norm(b) > TOL:    # stopping criterion based on Chapter 7, page 145      
        for i in range(n):
            s = 0
            for j in range(n):
                if j != i:
                    s += A[i,j] * x[j]
            x_new[i] = (b[i] - s) / A[i,i]
        x = x_new
        r = b - np.dot(A, x)
    return x


## Gauss-Seidel iteration
## Jacobi iteration
The code is based on the iteration from example 7.9 in the course book. As with the last iteration we use the stopping criterion from page 145. 

In [ ]:
def gauss_seidel_iteration(A, b):
    TOL = 1e-6
    n = np.shape(b)[0]
    x = np.zeros(n)
    x_new = np.zeros(n)

    r = b - np.dot(A, x)
    while np.linalg.norm(r) / np.linalg.norm(b) > TOL:  # stopping criterion based on Chapter 7, page 145    
        for i in range(n):
            s1 = 0
            s2 = 0
            for j in range(n):
                if j < i:
                    s1 += A[i,j] * x_new[j]
                if j > i:
                    s2 += A[i,j] * x[j]
            x_new[i] = (b[i] - s1 - s2) / A[i,i]
        x = x_new
        r = b - np.dot(A, x)
    return x


## Newton's method
This code is based on algortihm 8.2 in the course book. One possible problem with this algortihm is that is there are two solutions for example for a quadratic equation, it will only converge to one of the solutions, more if we start at an extreme point for $f$ we will divide by zero so the iteration wont terminate. From lecture 4 we have that the method will have quadratic convergence to the solution assuming that we start close to the solution and that $f$ is at least $C²$ continous.

In [ ]:
def newtons_method(f, df):
    x = 0.01
    TOL = 1e-6
    while np.abs(f(x)) > TOL:
        x = x - f(x) / df(x)
    return x

# **Results**

In this part we present tests on the code and their results.

## Jacobi iteration
We test the Jacobi iteration on a small system of equations. The residual norm and the distance from the real result is printed.

In [ ]:
def test_jacobi_iteration():
    A = np.array([[2, -1], [-1, 2]])
    b = np.array([1, 1])
    real_root = np.array([1, 1])
    x = jacobi_iteration(A, b)
    return np.linalg.norm(A.dot(x) - b), np.linalg.norm(x - real_root)

print(test_jacobi_iteration())

(3.5762786865234375e-07, 2.6656007498500226e-07)


## Gauss-Seidel iteration
We test the Gauss-Seidel iteration on a small system of equations. The residual norm and the distance from the real result is printed.

In [ ]:
def test_gauss_seidel_iteration():
    A = np.array([[2, -1], [-1, 2]])
    b = np.array([1, 1])
    real_root = np.array([1, 1])
    x = gauss_seidel_iteration(A, b)
    return np.linalg.norm(A.dot(x) - b), np.linalg.norm(x - real_root)

print(test_gauss_seidel_iteration())

(7.152557373046875e-07, 5.331201499700045e-07)


## Newton's method
We test newtond method on $x^2 -2 = 0$ with $f(x) = x^2 - 2$ and $f'(x) = 2x$. $|f(x)|$ where $x$ is the result and the distance from the real result is printed.

In [ ]:
def test_newtons_method():
    f = lambda x: x**2 - 2
    df = lambda x: 2*x
    real_root = np.sqrt(2)
    x = newtons_method(f, df)
    return np.abs(f(x)), np.abs(x - real_root)

print(test_newtons_method())

(2.1103119252074976e-12, 7.460698725481052e-13)


# **Discussion**
The code seem to work on the tests. In real code in an application I would probably also include checks to see that the convergence criterion is met for the first two iterations. Newtonds method has a pretty obvoius weakness which is that its converge depends on the function and starting point. So some care has to be taken into account when choosing starting point.
